In [8]:
import pickle
import numpy as np

from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

SEED = 42

In [9]:
model_type = 'glove-twitter-200'
wv_num = 200
wv_path = ['./data/train_data/train', './data/test_data/test']

with open(wv_path[0] + f'_{model_type}.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)
with open(wv_path[1] + f'_{model_type}.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

In [10]:
X_train, y_train = X_train[:20000], y_train[:20000]
X_test, y_test = X_test[:5000], y_test[:5000]

In [11]:
print(len(X_test[1]))
print(y_test[1])

4
sadness


In [12]:
# data process
max_length = max(len(i) for i in X_test + X_train)

def preprocess_dataset(X):
    X_list = []
    for i in X:
        temp = i
        for _ in range(max_length - len(i)):
            temp.append(np.zeros(wv_num))
        np_array = np.array(temp)
        trans = np.transpose(np_array)
        X_list.append(torch.from_numpy(trans))
    X_tensor = torch.stack(X_list)
    return X_tensor

X_train_tensor = preprocess_dataset(X_train)
print(f"Shape of training data {[X_train_tensor.shape[i] for i in range(3)]}")
X_test_tensor = preprocess_dataset(X_test)
print(f"Shape of training data {[X_test_tensor.shape[i] for i in range(3)]}")

Shape of training data [20000, 200, 106]
Shape of training data [5000, 200, 106]


In [13]:
# process labels
labels = list(set(y_train + y_test))
label_num = len(labels)
for i in range(len(y_train)):
    y_train[i] = labels.index(y_train[i])
for i in range(len(y_test)):
    y_test[i] = labels.index(y_test[i])
print(f"Number of label types: {label_num}")

Number of label types: 12


In [14]:
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X =torch.tensor(X, dtype=torch.float)
        self.y =torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

def initialize_loader(X_train_tensor, X_test_tensor, y_train, y_test):
    # no need to do the scale since original wv already did
    train_data = CustomDataset(X_train_tensor, y_train)
    test_data = CustomDataset(X_test_tensor, y_test)

    # convert to DataLoader for batch processing and shuffling
    train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
    return train_dataloader, test_dataloader

train_dataloader, test_dataloader = initialize_loader(X_train_tensor, X_test_tensor, y_train, y_test)

C:\Users\Chihao Shen\AppData\Local\Temp\ipykernel_23280\511967321.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X =torch.tensor(X, dtype=torch.float)


In [15]:
# model
class CNN(nn.Module):
    def __init__(self, dim_in, dim_conv, dim_out, dropout_rate=0.5, l2_norm=3):
        super(CNN, self).__init__()
        self.conv1_3 = nn.Conv1d(dim_in, dim_conv, 3, padding=5)  # 33
        self.conv1_4 = nn.Conv1d(dim_in, dim_conv, 4, padding=5)  # 32
        self.conv1_5 = nn.Conv1d(dim_in, dim_conv, 5, padding=5)  # 31
        self.bn = nn.BatchNorm1d(dim_conv * 3)
        self.ReLU = nn.ReLU()
        self.maxpool_1 = nn.MaxPool1d(kernel_size=max_length+8)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=max_length+7)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=max_length+6)
        self.fc = nn.Linear(dim_conv * 3, dim_out)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.softmax = nn.Softmax()


    def forward(self, x):
        x1 = self.conv1_3(x)
        x1 = self.ReLU(x1)
        x1 = self.maxpool_1(x1)


        x2 = self.conv1_4(x)
        x2 = self.ReLU(x2)
        x2 = self.maxpool_2(x2)

        x3 = self.conv1_5(x)
        x3 = self.ReLU(x3)
        x3 = self.maxpool_3(x3)

        x = torch.cat((x1, x2, x3), dim=1)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x


In [16]:
torch.manual_seed(SEED)
dim_in = wv_num
dim_conv = 100
dim_out = label_num
lr = 0.001
model = CNN(dim_in, dim_conv, dim_out)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
# training
max_norm = 3
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    optimizer.zero_grad()
    train_loss, correct_num = 0, 0
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()

        for name, module in model.named_modules():
            if isinstance(module, nn.Linear):
                for name, param in module.named_parameters():
                    param_norm = param.data.norm(2)
                    if param_norm > max_norm:
                        param.data.mul_(max_norm / (param_norm + 1e-6))

        optimizer.step()
        model.eval()
        with torch.no_grad():
            pred = model(X)
            loss = loss_fn(pred, y)
            train_loss += loss.item() * X.size(0)
            correct_num += (torch.eq(torch.argmax(pred, dim=1), y)).type(torch.float).sum().item()

    train_loss /= size
    train_acc = correct_num / size
    return train_loss, train_acc

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct_num = 0, 0
    model.eval()  # inform no dropout and fix bn during testing

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item() * X.size(0)
            correct_num += (torch.eq(torch.argmax(pred, dim=1), y)).type(torch.float).sum().item()

    test_loss /= size
    test_acc = correct_num / size
    return test_loss, test_acc
        

In [18]:
train_loss_, train_acc_, test_loss_, test_acc_ = [], [], [], []
no_epochs = 100

# start training
for epoch in range(no_epochs):
    train_loss, train_acc = train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loss, test_acc = test_loop(test_dataloader, model, loss_fn)

    train_loss_.append(train_loss), train_acc_.append(train_acc)
    test_loss_.append(test_loss), test_acc_.append(test_acc)

    print(f"Epoch {epoch+1}, train_loss {train_loss:>7f} train_acc {train_acc:>4f}, test_loss {test_loss:>7f}, test_acc {test_acc:>4f}")

d:\Program Files\Python310\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch 1, train_loss 2.364616 train_acc 0.259950, test_loss 2.331489, test_acc 0.289800
Epoch 2, train_loss 2.259337 train_acc 0.387000, test_loss 2.323881, test_acc 0.295800
Epoch 3, train_loss 2.199672 train_acc 0.456700, test_loss 2.319931, test_acc 0.296000
Epoch 4, train_loss 2.152162 train_acc 0.511100, test_loss 2.317776, test_acc 0.303000
Epoch 5, train_loss 2.114032 train_acc 0.548600, test_loss 2.319568, test_acc 0.303600
Epoch 6, train_loss 2.084501 train_acc 0.575600, test_loss 2.319477, test_acc 0.297600
Epoch 7, train_loss 2.058583 train_acc 0.598200, test_loss 2.321835, test_acc 0.296800
Epoch 8, train_loss 2.038628 train_acc 0.613300, test_loss 2.319312, test_acc 0.297000
Epoch 9, train_loss 2.020327 train_acc 0.627750, test_loss 2.323617, test_acc 0.287600
Epoch 10, train_loss 2.005532 train_acc 0.638150, test_loss 2.326016, test_acc 0.287600
Epoch 11, train_loss 1.992729 train_acc 0.650000, test_loss 2.320407, test_acc 0.292600
Epoch 12, train_loss 1.981796 train_acc 0